In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

In [2]:
data_path = '../data/CALC_SEG/'
calcSEG = [f for f in os.listdir(data_path) if f.endswith('.csv')]
output_path = '../data/CALC_STRAIGHT/'
fig_path = '../data/CALC_TURNS_FIG/'

In [3]:
X = pd.read_csv('../data/PN chuze/PN ataxie.csv')
X.head()

,File,Ts [s],fs [Hz],Frames,Type,od,do,od.1,do.1,od.2,...,od.6,do.6,od.7,do.7,od.8,do.8,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,N-01-1-Char00,0.008,125.0,30060.0,norma,38314,40368,41497.0,69503.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A-03-1-Char00,0.008,125.0,10482.0,ataxie,40200,41032,42822.0,46285.0,47286.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N-04-1-Char00,0.008,125.0,24964.0,norma,28500,30879,32283.0,34970.0,36292.0,...,52700.0,61820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N-06-1-Char00,0.008,125.0,23941.0,norma,30120,32045,34437.0,36508.0,38509.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N-07-1-Char00,0.008,125.0,34820.0,norma,28900,63720,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
text_file = open('../data/JointNamesCalc.txt', "r")
joint_names_calc = text_file.read().split('\n')
len(joint_names_calc)

61

In [5]:
corpus_joints = ['Head','Neck','Spine3','Spine2','Spine1','Spine']
corpus_joints_ind = np.zeros([len(corpus_joints),1])
cc = 0
for c in corpus_joints:
    corpus_joints_ind[cc] = joint_names_calc.index(c) + 1
    cc+=1
corpus_joints_ind

array([[16.],
       [17.],
       [18.],
       [19.],
       [20.],
       [21.]])

In [6]:
def pos_plot(x,xf,xfr,i,pos):
    ax[pos].plot(time,x[:,i])
    ax[pos].plot(time,xf[:,i],'r')
    ax[pos].plot(time,-x[:,i],'k')
    ax[pos].plot(time,xfr[:,i],'g')

In [7]:
segs = pd.DataFrame(columns=['subjectID','type','part','t_sampling','sample_len','time_len'])

for seg in calcSEG:
    subject =  seg.split(sep='Char00',maxsplit=1)[0]+'Char00'
    part = seg.split(sep='Char00_',maxsplit=1)[1][:-4]
    print(subject + ':' + part)
    
    df = pd.read_csv(data_path + seg)
    N = np.size(df,axis=0)
    # Check for the T_sampling from bvh file
    T_sampling = X.loc[X['File'] == subject,'Ts [s]'].values
    # Create time vector for this part
    time = np.linspace(0,N,num=N)*T_sampling
    
    # TURNS IN PART ?
    if time[-1] > 30:
        
        # Take column with position only
        col_list = df.columns.to_list()
        position_cols = [s for s in col_list if "X" in s]
        xdf = df.filter(position_cols,axis=1)
    
        # CORPUS
        corpus_cols_ind = []
        for c in corpus_joints_ind:
            for col in xdf.columns.to_list():
                if (str(int(c))) in col:
                    corpus_cols_ind.append(xdf.columns.to_list().index(col))

        corpus = xdf.filter(xdf.columns[corpus_cols_ind],axis=1)
        corpus_cols = corpus.columns.to_list()

        xdim_cols = [s for s in corpus_cols if "x" in s]
        ydim_cols = [s for s in corpus_cols if "y" in s]
        zdim_cols = [s for s in corpus_cols if "z" in s]
        x = np.zeros([N,3])
        x[:,0] = corpus.filter(xdim_cols,axis=1).mean(axis=1).to_numpy()
        x[:,1] = corpus.filter(ydim_cols,axis=1).mean(axis=1).to_numpy()
        x[:,2] = corpus.filter(zdim_cols,axis=1).mean(axis=1).to_numpy()        
        
        xf = np.zeros_like(x)
        xfr = np.zeros_like(x)

        w_len = np.min((1001,N))
        if (w_len % 2) == 0: w_len -= 1

        p_x = np.empty(0)
        p_y = np.empty(0)

        n_peaks = np.empty([np.size(x,axis=1)])

        fig,ax = plt.subplots(3,1,figsize = (9,6))
        for i in range(0,np.size(x,axis=1)):
            xf[:,i] = savgol_filter(x[:,i],w_len,2) 
            xfr[:,i] = savgol_filter(-x[:,i],w_len,2)
            pos_plot(x,xf,xfr,i,i)
            ax[0].set_title('File:' + seg + ' Movement: Mean of corpus joints')
            ax[0].set_ylabel('x')
            ax[1].set_ylabel('y')
            ax[2].set_ylabel('z')
            ax[2].set_xlabel('Time [s]')

            peaks, _ = find_peaks(xf[:,i],prominence=0.2, width=20, distance=10/T_sampling)
            ax[i].plot(time[peaks],xf[peaks,i],'ms')
            
            peaks_r, _ = find_peaks(xfr[:,i],prominence=0.2, width=20, distance=10/T_sampling)
            ax[i].plot(time[peaks_r],xfr[peaks_r,i],'mo')
            
            n_peaks[i] = len(peaks) + len(peaks_r)
            
            if i == 0:
                p_x = np.concatenate((peaks,peaks_r),axis=None)
                p_x.sort()
            if i == 1:
                p_y = np.concatenate((peaks,peaks_r),axis=None)
                p_y.sort()                
        n_turns = max([n_peaks[0],n_peaks[1]])
            
        if n_peaks[0] > n_peaks[1]:
            turns = p_x
        else:
            turns = p_y
                
            #turns = np.concatenate((p,pr),axis=None)
            #with open(output_path + subject + '_' +  part + '.txt','w') as f:
            #     np.savetxt(f,turns,fmt='%d')
            
        plt.savefig(fig_path + subject + "_" + part + '.png')
        plt.close(fig)  
    else:
        n_turns = 0
        n_peaks = np.array([0,0])   
    thisDF = pd.DataFrame(data={'subjectID':subject,'type':X.loc[X['File'] == subject,'Type'],
                        'part':part,'t_sampling':T_sampling,'sample_len':N,
                        'time_len':N*T_sampling,'n_turns': n_turns,'n_peaks_Y': n_peaks[1],'n_peaks_X': n_peaks[0],})
        
    segs = pd.concat([segs, thisDF],ignore_index=True,sort=False)
    df.rename(columns={"Unnamed: 0": "OrigIndex"},inplace=True)
    if n_turns == 0:
        if thisDF.time_len.values > 5:
            df.to_csv(output_path + subject + part + 'seg-0.csv',index=False)
        else:
            print('<5s part of record: ' + subject + part)
    else:
        segSTRAIGHT = 0
        startSTRAIGHT = 0
        turns = np.append(turns,N)
        for turn in turns:
            thisSTRAIGHT = df.loc[startSTRAIGHT:turn,:]
            thisSTRAIGHT_len = turn*T_sampling - startSTRAIGHT*T_sampling 
            if thisSTRAIGHT_len > 5:
                thisSTRAIGHT.to_csv(output_path + subject + part + 'seg-' + str(segSTRAIGHT) + '.csv',index=False)
                segSTRAIGHT += 1
            else:
                print('<5s segment: ' + subject + part + 'index between: ' + str(startSTRAIGHT) + '-' + str(turn))
            startSTRAIGHT = turn

N-04-1-Char00:part6
N-920828-Char00:part4
N-920828-Char00:part5
N-04-1-Char00:part7
N-04-1-Char00:part5
N-920828-Char00:part7
N-920828-Char00:part10
N-16-2-Char00:part4
N-07-1-Char00:part1
N-16-2-Char00:part5
N-920828-Char00:part11
N-920828-Char00:part6
N-04-1-Char00:part4
N-920828-Char00:part2
N-16-2-Char00:part1
N-920828-Char00:part3
N-04-1-Char00:part1
N-04-1-Char00:part3
N-920828-Char00:part1
N-16-2-Char00:part2
N-16-2-Char00:part3
N-04-1-Char00:part2
A-766012-Char00:part1
A-13-1-Char00:part1
<5s part of record: A-13-1-Char00part1
A-03-1-Char00:part1
A-13-1-Char00:part2
<5s part of record: A-13-1-Char00part2
A-03-1-Char00:part3
A-615829-Char00:part1
A-03-1-Char00:part2
A-681001-Char00:part1
A-13-1-Char00:part3
N-01-1-Char00:part1
A-13-1-Char00:part7
A-13-1-Char00:part6
N-01-1-Char00:part2
<5s segment: N-01-1-Char00part2index between: 27858-28007
A-13-1-Char00:part4
<5s part of record: A-13-1-Char00part4
N-15051994-Char00:part8
A-03-1-Char00:part5
A-03-1-Char00:part4
A-13-1-Char00:p

In [8]:
segs

,subjectID,type,part,t_sampling,sample_len,time_len,n_turns,n_peaks_Y,n_peaks_X
0,N-04-1-Char00,norma,part6,0.008,2613,20.904,0.0,0.0,0.0
1,N-920828-Char00,norma,part4,0.017,761,12.937,0.0,0.0,0.0
2,N-920828-Char00,norma,part5,0.017,712,12.104,0.0,0.0,0.0
3,N-04-1-Char00,norma,part7,0.008,9121,72.968,4.0,4.0,4.0
4,N-04-1-Char00,norma,part5,0.008,1974,15.792,0.0,0.0,0.0
5,N-920828-Char00,norma,part7,0.017,795,13.515,0.0,0.0,0.0
6,N-920828-Char00,norma,part10,0.017,1143,19.431,0.0,0.0,0.0
7,N-16-2-Char00,norma,part4,0.017,729,12.393,0.0,0.0,0.0
8,N-07-1-Char00,norma,part1,0.008,34821,278.568,19.0,19.0,19.0
9,N-16-2-Char00,norma,part5,0.017,771,13.107,0.0,0.0,0.0


In [9]:
segs.to_csv('../data/SEGMENTS.csv',index=False)